# NRC-30 NIRCam Coronagraphic Target Acquisition

## PID 775 - Partial Rehearsal Version

This activity is a two-part test of NIRCam coronagraphic target acquisition (TA).  Part 1 determines the precise offsets required to move a star from its measured position in the target-acquisition image to the ideal location behind each occulter.   Part 2 follows the analysis period for Part 1 and tests the implementation of the derived offsets. Both parts follow the successful completion of CAR-NIRCAM-021b (Coronagraphic Astrometric Calibration), the goal of which is a geometric distortion map of the coronagraphic FOV with RMS errors less than 3 mas in each axis.

In [1]:
# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# Enable inline plotting at lower left
%matplotlib inline

# Load progress bars
from tqdm.auto import trange, tqdm

In [2]:
import pynrc
from pynrc import nrc_utils, obs_hci, nrc_hci
from pynrc.simul.ngNRC import create_level1b_FITS

# Disable informational messages and only include warnings and higher
pynrc.setup_logging(level='WARN')

pyNRC log messages of level WARN and above will be shown.
pyNRC log outputs will be directed to the screen.


In [3]:
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
from astropy.time import Time
from astroquery.irsa import Irsa

# APT Inputs

From the final APT file, we need to export a number of files, including the `.timing.json`, `.smart_accounting`, `.pointing`, and `.xml` files. These are then parsed by `pynrc` to configure a series of observation visits and associated NIRCam objects.

In [4]:
# Read in APT
pid = 775
pid_str = f'pid{pid:05d}'

save_dir = f'/Users/jarron/NIRCam/Data/NRC_Sims/Sim_{pid_str}/'

# APT input files
apt_file_dir = '../../APT_output/'
fprefix = f'pid{pid}_LRE6'
json_file     = f'{apt_file_dir}{fprefix}.timing.json'
sm_acct_file  = f'{apt_file_dir}{fprefix}.smart_accounting'
pointing_file = f'{apt_file_dir}{fprefix}.pointing'
xml_file      = f'{apt_file_dir}{fprefix}.xml'

## Target Information

For each target specified in the APT file, we want to populate a dictionary with coordinate information and astrophysical properties. The dictionary keys should match the APT "Name in the Proposal" so that 

For each designated target, there are four types of objects that can be added: stellar source, point source companions, disk object, and/or a table of point sources specifying magnitudes in NIRCam filters.

1. Stellar source:
``` python
    params_star = {
        'sptype' : 'A0V', 'Teff': 10325, 'metallicity' : 0, 'log_g' : 4.09,
        'v_mag' : 5.690, 'j_mag': 5.768, 'h_mag': 5.753, 'k_mag': 5.751,
    }
```
2. Companions (where `bp_renorm` is a `pysynphot` bandpass):
``` python
    params_hci_companions = {
        'b' : {'xy':(-1.625,  0.564), 'runits':'arcsec', 'renorm_args':(16.0, 'vegamag', bp_renorm)},
        'c' : {'xy':( 0.319,  0.886), 'runits':'arcsec', 'renorm_args':(15.0, 'vegamag', bp_renorm)},
        'd' : {'xy':( 0.588, -0.384), 'runits':'arcsec', 'renorm_args':(14.6, 'vegamag', bp_renorm)},
        'e' : {'xy':( 0.249,  0.294), 'runits':'arcsec', 'renorm_args':(14.7, 'vegamag', bp_renorm)},
    }
```
3. Companion disk model:
``` python
    params_disk_model = {
        'file': 'HD10647.fits',  # path to file
        'wavelength': 3.0,       # model wavelength (um)
        'pixscale': 0.01575,     # input image arcsec/pixel
        'units': 'Jy/pixel',     # model flux units (e.g., mJy/arcsec, Jy/pixel, etc)
        'dist': 17.34,           # assumed distance to model (pc)
        'cen_star': True,        # does the model include stellar flux?
    }
```
4. Table of stellar sources:
``` python
    from astropy.io import ascii
    
    cat_file = 'lmc_catalog_flag1.cat'
    names = [
        'index', 'ra', 'dec', 'F070W', 'F090W', 'F115W', 'F140M', 'F150W', 'F150W2',
        'F162M', 'F164N', 'F182M', 'F187N', 'F200W', 'F210M', 'F212N', 'F250M', 'F277W', 
        'F300M','F322W2', 'F323N', 'F335M', 'F356W', 'F360M', 'F405N', 'F410M', 'F430M', 
        'F444W', 'F460M', 'F466N', 'F470N', 'F480M'
    ]

    src_tbl = ascii.read(cat_file, names=names)
```

In [5]:
# Define 2MASS Ks bandpass and source information
bp_k = pynrc.bp_2mass('k')

# Science    source,  dist, age, sptype, Teff, [Fe/H], log_g, mag, band
args_sci = ('HR 8799', 39.0, 30,  'F0V', 7430, -0.47, 4.35, 5.24, bp_k)

targ_dict = {}
targ_dict['HD-42525'] = {
    'type' : 'FixedTargetType',
    'TargetName' : 'HD-42525', 'TargetArchiveName' : 'eta01 Dor',
    'EquatorialCoordinates' : "06 06 9.3818 -66 02 22.64",
    'RAProperMotion'  : 13.656*u.mas/u.yr, 
    'DecProperMotion' : 27.815*u.mas/u.yr, 
    'parallax' : 9.7251*u.mas, 'age_Myr' : 100,
    'params_star' : {
        'sptype' : 'A0V', 'Teff': 10325, 'metallicity' : 0, 'log_g' : 4.09,
        'v_mag' : 5.690, 'j_mag': 5.768, 'h_mag': 5.753, 'k_mag': 5.751,
    },
    'params_companions' : None,
    'params_disk_model' : None,
    'src_tbl' : None,
}

# Populate coordinate and stellar spectrum objects
for k in targ_dict.keys():
    d = targ_dict[k]
    dist = Distance(parallax=d['parallax']) if d['parallax'] is not None else None
    c = SkyCoord(d['EquatorialCoordinates'], frame='icrs', unit=(u.hourangle, u.deg),
                 pm_ra_cosdec=d['RAProperMotion'], pm_dec=d['DecProperMotion'],
                 distance=dist, obstime='J2000')
    d['sky_coords'] = c
    d['ra_J2000'], d['dec_J2000'] = (c.ra.deg, c.dec.deg)
    d['dist_pc'] = c.distance.value if dist is not None else None
    
    dstar = d['params_star']
    if dstar is not None:
        # Option to add WISE values
        # table = Irsa.query_region(c, catalog="allwise_p3as_psd", spatial="Cone", radius=5*u.arcsec)
        # dstar['bp_w1'], dstar['bp_w2'], dstar['bp_w3'], dstar['bp_w4'] = \
        #     table[0]['w1mpro', 'w2mpro', 'w3mpro', 'w4mpro']

        # Create stellar spectrum normalized to the K-Band magnitude
        kwargs = {kw:d.get(kw) for kw in ['Teff', 'metallicity', 'log_g']}
        dstar['sp'] = pynrc.stellar_spectrum(dstar['sptype'], dstar['k_mag'], 'vegamag', bp_k, **kwargs)

In [61]:
sim_config = {
    # APT input files
    'json_file'     : json_file,
    'sm_acct_file'  : sm_acct_file,
    'pointing_file' : pointing_file,
    'xml_file'      : xml_file,
    # Output directory
    'save_dir'      : save_dir,
    
    # Date and time of observations
    'obs_date' : '2022-03-01', 
    'obs_time' : '12:00:00',
    # Position angle of observatory
    # User should check acceptable range in APT's Roll Analysis
    'pa_v3'    : 298.5,
    
    # Source information
    'params_targets' : targ_dict,
    
    # PSF size information for WebbPSF_ext
    'params_webbpsf' : {'fov_pix': None, 'oversample': 2},
    # Position-dependent PSFs for convolution
    'params_psfconv' : {'npsf_per_full_fov': 5, 'osamp': 1, 'sptype': 'G0V'},
    # Wavefront error drift settings
    'enable_wfedrift' : False,
    # For coronagraphic masks, sample large grid of points?
    'large_grid' : True,
    
    # Slew and dither pointing uncertainties
    'large_slew' : 100.0,   # Slew to target (mas)
    'ta_sam'     : 5.0,     # SAM movements from TA position (mas)
    'std_sam'    : 5.0,     # Standard dither values (mas)
    'sgd_sam'    : 2.5,     # Small grid dithers (mas)
    
    # Type of image files to save
    'save_slope' : False,  # Save ideal noiseless slope images to FITS
    'save_dms'   : False,   # Save DMS-like ramps to FITS
    'dry_run'    : False,  # Perform a dry-run, not generating any data, just printing visit info
    
    # Initialize random seeds if repeatability is required
    # Create separate random number generators for dithers and noise
    'rand_seed_init'  : 1234,

    # Ramp parameters
    'params_noise' : {
        'include_poisson'   : True,     # Photon Noise
        'include_dark'      : True,     # Dark current
        'include_bias'      : True,     # Bias image offset
        'include_ktc'       : True,     # kTC Noise
        'include_rn'        : True,     # Read Noise
        'include_cpink'     : True,     # Correlated 1/f noise between channel
        'include_upink'     : True,     # Channel-dependent 1/f noise
        'include_acn'       : True,     # Alternating column noise
        'apply_ipc'         : True,     # Interpixel capacitance
        'apply_ppc'         : True,     # Post-pixel coupling
        'amp_crosstalk'     : True,     # Amplifier crosstalk
        'include_refoffsets': True,     # Reference offsets
        'include_refinst'   : True,     # Reference pixel instabilities
        'include_colnoise'  : True,     # Transient detector column noise
        'add_crs'           : True,     # Include cosmic ray
        'cr_model'          : 'SUNMAX', # Cosmic ray model ('SUNMAX', 'SUNMIN', or 'FLARES')
        'cr_scale'          : 1,        # Cosmic ray probabilities scaling
        'apply_nonlinearity': True,     # Apply non-linearity
        'random_nonlin'     : True,     # Add randomness to non-linearity
        'apply_flats'       : True,     # pixel-to-pixel QE variations and field-dep illum
    },
}

In [39]:
for vid in ['032:001', '033:001', '034:001','035:001']:
    create_level1b_FITS(sim_config, dry_run=True, save_dms=False, visit_id=vid)

Obs Params:   0%|          | 0/20 [00:00<?, ?it/s]

Exposures:   0%|          | 0/3 [00:00<?, ?it/s]

NRCA4 NRCA4_FULL_MASKSWB_F210M F210M HD-42525 032:001 03102 1 (+0.141, -0.002)
NRCA4 NRCA4_FULL_MASKSWB_F210M F210M HD-42525 032:001 03102 2 (+0.141, +0.010)
NRCA4 NRCA4_FULL_MASKSWB_F210M F210M HD-42525 032:001 03102 3 (+0.138, -0.019)


Obs Params:   0%|          | 0/20 [00:00<?, ?it/s]

Exposures:   0%|          | 0/5 [00:00<?, ?it/s]

...

NRCA5 NRCA5_FULL_TAMASKLWB F335M HD-42525 035:001 03102 1 (-0.074, +0.025)


Exposures:   0%|          | 0/4 [00:00<?, ?it/s]

NRCA5 NRCA5_MASKLWB_F335M F335M HD-42525 035:001 03106 1 (-0.006, +0.005)


Exposures:   0%|          | 0/4 [00:00<?, ?it/s]

NRCA5 NRCA5_TAMASKLWB F335M HD-42525 035:001 02102 1 (-0.074, +0.025)


In [62]:
create_level1b_FITS(sim_config, save_slope=False, save_dms=True, visit_id='035:001', apname='NRCA5_TAMASKLWB')

Obs Params:   0%|          | 0/20 [00:00<?, ?it/s]

PPC Amps:   0%|          | 0/4 [00:00<?, ?it/s]

Frames:   0%|          | 0/1 [00:00<?, ?it/s]

Frames:   0%|          | 0/1 [00:00<?, ?it/s]

Frames:   0%|          | 0/1 [00:00<?, ?it/s]

...

Ramps:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

Uncorr 1/f:   0%|          | 0/1 [00:00<?, ?it/s]

ACN:   0%|          | 0/1 [00:00<?, ?it/s]

215.217099867727 49917
(12, 64, 64) (12, 64, 64) 12 2 3 64 64 (0, 64) (0, 64)
49917
  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/pynrc_jw00775035001_02102_00001_nrca5_uncal.fits


In [29]:
nrc = pynrc.NIRCam(filter='F335M', pupil='WEDGELYOT', ND_acq=True)

In [32]:
nrc.update_detectors(read_mode='SHALLOW2', ngroup=3, xpix=64, ypix=64, verbose=True)

New Ramp Settings
  read_mode  : SHALLOW2
  nf         :        2
  nd2        :        3
  ngroup     :        3
  nint       :        1
New Detector Settings
  wind_mode  :   WINDOW
  xpix       :       64
  ypix       :       64
  x0         :      280
  y0         :     1525
New Ramp Times
  t_group    :    0.251
  t_frame    :    0.050
  t_int      :    0.602
  t_int_tot1 :    0.655
  t_int_tot2 :    0.000
  t_exp      :    0.602
  t_acq      :    0.657


In [35]:
nrc.sensitivity(sp=sp, forwardSNR=True, units='vegamag')

({'type': 'Point Source',
  'snr': 13.441202679563103,
  'Spectrum': 'A0V',
  'flux': 5.754118355408011,
  'flux_units': 'vegamag'},
 {'type': 'Surface Brightness',
  'snr': 31.90824018823915,
  'Spectrum': 'A0V',
  'flux': 5.754118355408011,
  'flux_units': 'vegamag/arcsec^2'})

In [25]:
sp = targ_dict['HD-42525']['params_star']['sp']
im = nrc.calc_psf_from_coeff(sp=sp, return_hdul=False, return_oversample=False)

In [23]:
im.max()

243.25840937138082

In [26]:
im.max()

125898.59772764967

In [28]:
243.25840937138082 / 125898.59772764967

0.0019321772741076112

In [9]:
for vid in ['032:001', '033:001', '034:001','035:001']:
    create_level1b_FITS(sim_config, save_slope=False, save_dms=True, visit_id=vid)

Obs Params:   0%|          | 0/20 [00:00<?, ?it/s]

PPC Amps:   0%|          | 0/4 [00:00<?, ?it/s]

Frames:   0%|          | 0/1 [00:00<?, ?it/s]

Frames:   0%|          | 0/1 [00:00<?, ?it/s]

Frames:   0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/14 [00:00<?, ?it/s]

Non-Linearity:   0%|          | 0/12 [00:00<?, ?it/s]

Uncorr 1/f:   0%|          | 0/1 [00:00<?, ?it/s]

ACN:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: cannot reshape array of size 0 into shape (1,64,64)

In [9]:
create_level1b_FITS(sim_config, save_slope=True, save_dms=False)#, visit_id='032:001')

Obs Params:   0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775034001_03104_00001_nrca2_uncal.fits
  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775034001_03105_00001_nrca2_uncal.fits


  0%|          | 0/4 [00:00<?, ?it/s]

  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775034001_03102_00001_nrca2_uncal.fits


...

  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775033001_03102_00001_nrca5_uncal.fits
  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775033001_03102_00002_nrca5_uncal.fits
  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775033001_03102_00003_nrca5_uncal.fits
  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775033001_03102_00004_nrca5_uncal.fits
  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775033001_03102_00005_nrca5_uncal.fits


  0%|          | 0/8 [00:00<?, ?it/s]

  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775035001_03106_00001_nrca5_uncal.fits
  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775035001_03106_00002_nrca5_uncal.fits
  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775035001_03106_00003_nrca5_uncal.fits
  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775035001_03106_00004_nrca5_uncal.fits
  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775035001_03106_00005_nrca5_uncal.fits


  0%|          | 0/8 [00:00<?, ?it/s]

  Saving: /Users/jarron/NIRCam/Data/NRC_Sims/Sim_pid00775/slope_jw00775035001_02102_00001_nrca5_uncal.fits


In [10]:
from pynrc.simul.apt import DMS_input
obs_input = DMS_input(xml_file, pointing_file, json_file, sm_acct_file)

visit = obs_input.program_info['034:001']

obs_params_all = obs_input.gen_all_obs_params()

Obs Params:   0%|          | 0/20 [00:00<?, ?it/s]

In [11]:
visit

{'obs_num': 34,
 'visit_num': 1,
 'visit_start': 0,
 'slew_duration': 1800,
 'visit_duration': 2002,
 'exp_start_times': array([2926, 3094, 3343, 3566]),
 'PrimaryDitherType': array(['NONE', 'NONE', 'NONE', 'NONE'], dtype='<U4'),
 'PrimaryDithers': array(['NONE', '2', '2', '1'], dtype='<U4'),
 'DitherSize': array(['NONE', 'NONE', 'NONE', 'NONE'], dtype='<U4'),
 'SubpixelPositions': array(['NONE', 'NONE', 'NONE', '1'], dtype='<U4'),
 'SubpixelDitherType': array(['NONE', 'NONE', 'NONE', 'NONE'], dtype='<U17'),
 'SmallGridDitherType': array(['NONE', 'NONE', 'NONE', 'NONE'], dtype='<U11'),
 'DitherPatternType': array(['NONE', 'NONE', 'NONE', 'NONE'], dtype='<U4'),
 'ImageDithers': array(['NONE', '2', '2', 'NONE'], dtype='<U4'),
 'number_of_dithers': array([1, 2, 2, 1]),
 'rand_seed_dith': 3473547997,
 'rand_seed_noise': 67684538,
 'TargetID': array(['HD-42525', 'HD-42525', 'HD-42525', 'HD-42525'], dtype='<U8'),
 'aperture': array(['NRCA2_TAMASK210R', 'NRCA2_FULL_TAMASK210R', 'NRCA2_FULL_MA

In [10]:
obs_params = obs_params_all[7]
obs_params['obs_id_info']

{'program_number': '00775',
 'observation_number': '033',
 'visit_number': '001',
 'visit_group': '03',
 'sequence_id': '1',
 'activity_id': '02',
 'exposure_number': '00004',
 'visit_id': '00775033001',
 'obs_id': 'V00775033001P0000000003102'}

In [11]:
obs_params

{'pi_name': 'Julien Girard',
 'title': 'NIRCam Target Acquisition for Coronagraphy (CAR NIRCam-30)',
 'category': 'COM',
 'sub_category': 'NIRCAM',
 'science_category': 'Exoplanets and Exoplanet Formation',
 'target_name': 'HD-42525',
 'catalog_name': 'UNKNOWN',
 'ra': 91.53909,
 'dec': -66.03962,
 'pa_v3': 69.47239167892067,
 'roll_offset': 0,
 'solar_elong': 90.75865992290616,
 'pitch_ang': 0.7586599229061619,
 'siaf_ap': <pysiaf.Aperture object AperName=NRCA5_MASK335R >,
 'ra_obs': 91.53909602925607,
 'dec_obs': -66.03961464024694,
 'siaf_ap_ref': <pysiaf.Aperture object AperName=NRCA5_MASK335R >,
 'ra_ref': 91.53909602925607,
 'dec_ref': -66.03961464024694,
 'date-obs': '2022-03-01',
 'time-obs': '12:00:00.000',
 'obs_id_info': {'program_number': '00775',
  'observation_number': '033',
  'visit_number': '001',
  'visit_group': '03',
  'sequence_id': '1',
  'activity_id': '02',
  'exposure_number': '00004',
  'visit_id': '00775033001',
  'obs_id': 'V00775033001P0000000003102'},
 'ob